In [1]:
import os
import csv
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import argparse
import ntpath
import sys
import tensorflow as tf
import numpy as np
import datetime
from prettytable import PrettyTable
import pandas as pd

from preprocessing import is_number

In [2]:
path = '/home/wanli/data/Extended_ctr/citeulike_a_extended/paper_info.csv'
null_token = 'NaN'
now = datetime.datetime.now()
    
clean_file_path = path +'.cleaned'


# Month converter
months = ['apr','aug', 'dec','feb', 'jan' ,'jul' ,'jun' ,'mar' ,'may', 'nov', 'oct', 'sep']
month_convert_func = lambda x: x if x in months else null_token

def number_convert_func (x):
    if is_number(x):
        return x
    else:
        print(x)
        return null_token
    
labels = ['doc_id', 'citeulike_id', 'type', 'pages', 'year']
labels_dtype = {'doc_id': np.int32, 'citeulike_id': np.int32, 'type': str, 'pages': np.int32}
convert_func= {'pages': number_convert_func, 'doc_id': number_convert_func,
               'citeulike_id': number_convert_func}
# labels = ['doc_id', 'citeulike_id', 'type', 'journal', 'booktitle', 'series', 'pages', 'year', 'month', 'address']
# labels_dtype = {'doc_id': np.int32, 'citeulike_id': np.int32, 'type': str, 'journal': str, 'booktitle': str,
#                 'series': str,
#                 'pages': np.int32, 'month': str, 'address': str}
# convert_func = {'month': month_convert_func, 'pages': number_convert_func, 'doc_id': number_convert_func,
#                 'citeulike_id': number_convert_func}

df = pd.read_table(clean_file_path, delimiter='\t', index_col = 'doc_id', usecols=labels,dtype=labels_dtype,
                     na_values='\\N',na_filter=False,
                     converters=convert_func)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: ParserWarning: Both a converter and dtype were specified for column doc_id - only the converter will be used
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: ParserWarning: Both a converter and dtype were specified for column citeulike_id - only the converter will be used
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: ParserWarning: Both a converter and dtype were specified for column pages - only the converter will be used


In [3]:
print(df.shape)
# count of unique values:
print (df.apply(pd.Series.nunique))

(16981, 4)
citeulike_id    16962
type               15
pages             263
year               88
dtype: int64


In [4]:
#Filter values with frequency less than min_freq
def filter(df, tofilter_list, min_freq):
    for col in tofilter_list:
        to_keep =  df[col].value_counts().reset_index(name="count").query("count > %d" %min_freq)["index"]
        to_keep = to_keep.values.tolist()
        df[col] = [x if x in to_keep else 'NaN' for x in df[col] ]
    return df

tofilter_list = []
df = filter(df,tofilter_list,2)
print(df.shape)
print (df.apply(pd.Series.nunique))

# 
# # Convert catigorical feature into one-hot encoding
# def dummmy_df(df, todummy_list):
#     for x in todummy_list:
#         dummies = pd.get_dummies(df[x], prefix=x, dummy_na=True)
#         df = df.drop(x,1)
#         df = pd.concat([df,dummies], axis=1)
#     return df

# todummy_list = ['type']
# df_dummy = dummmy_df(df,todummy_list)
# # print(df_dummy.head(2))
# # print (df2)
# print('----------------------------')
# # print (df_dummy['journal'].value_counts())
# # .reset_index(name="count").query("count > 3")["index"]
# print('----------------------------')
# print(df_dummy.shape)
# print (df_dummy.apply(pd.Series.nunique))

(16981, 4)
citeulike_id    16962
type               15
pages             263
year               88
dtype: int64


In [8]:
print (df.pages.unique())

['NaN' '4' '5' '8' '7' '12' '2' '3' '28' '10' '17' '9' '42' '203' '14' '11'
 '6' '20' '19' '18' '16' '56' '13' '29' '41' '39' '15' '40' '22' '31' '25'
 '23' '59' '27' '48' '33' '35' '34' '51' '30' '60' '43' '32' '24' '83' '1'
 '21' '70' '37' '46' '62' '53' '45' '26' '36' '38' '47' '44' '-1591'
 '-2546' '50' '74' '57' '-275' '49' '573' '-187' '-1825' '-9796' '-1585'
 '-87' '-374' '-395' '-96' '-86' '-284' '-93' '-681' '-387' '-134' '-476'
 '-696' '-1691' '-524' '-94' '-2389' '-177' '-595' '-2189' '-363' '-4495'
 '-292' '-12835' '-688' '-796' '-537' '-15515' '-1595' '-2114' '-191'
 '-144' '-391' '-297' '-10605' '-397' '-1196' '-1994' '-482' '-5935'
 '-2356' '-317' '-616' '-189' '-415' '-1089' '-57' '-693' '90' '54' '52'
 '63' '-385' '-64' '-1297' '55' '97' '65' '-789' '-280' '-173' '77' '-73'
 '-994' '-3289' '-433' '85' '-84' '261' '58' '66' '-388' '-1392' '108' '71'
 '-1070' '-370' '-1267' '-10906' '-88' '-194' '75' '68' '88' '-1896' '-163'
 '-95' '-12465' '-295' '-1940' '93' '73' '-845

In [5]:
print(df_dummy.shape)
df_dummy.dropna(inplace=True)
print(df_dummy.shape)
print (df_dummy.apply(pd.Series.nunique))

(16981, 19)
(16981, 19)
citeulike_id          16962
pages                   263
year                     88
type_NaN                  2
type_article              2
type_book                 2
type_booklet              2
type_electronic           2
type_inbook               2
type_incollection         2
type_inproceedings        2
type_manual               2
type_mastersthesis        2
type_misc                 2
type_phdthesis            2
type_proceedings          2
type_techreport           2
type_unpublished          2
type_nan                  1
dtype: int64


In [4]:
# min-max normalization:
from sklearn import preprocessing

# x = df.loc[:,todummy_list].values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df.loc[:,todummy_list] = pandas.DataFrame(x_scaled)

values = df.values
imputer = preprocessing.Imputer()
transformed_values = imputer.fit_transform(values)
# count the number of NaN values in each column
print(np.isnan(transformed_values).sum())

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(transformed_values)
transformed_df= pd.DataFrame(x_scaled)

0


In [5]:
print(transformed_df.shape)
print(x_scaled.shape)
# print(transformed_df.nunique())
# print (transformed_df.head(2))

(25976, 1893)
(25976, 1893)


In [13]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# training= np.random.randint(2, size=(x_scaled.shape[0],x_scaled.shape[1]+1))
# training[:,:-1] = x_scaled

y= np.random.randint(2, size=(x_scaled.shape[1]))
# y= y.reshape((y.shape[0],1))
print(x_scaled.shape)
print(y.shape)

# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, np.transpose(np.matrix(x_scaled)), np.transpose(y), cv=kfold, scoring='accuracy')
print(result.mean())

(25976, 1893)
(1893,)


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.491811938722
